In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

In [2]:
from ib_insync import *
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import math
import datetime 
import statsmodels.formula.api as smf 
from datetime import date, time, datetime, timedelta
import seaborn as sns
import random
from ta import add_all_ta_features
from ta.utils import dropna
from ta.trend import *
from collections import deque, defaultdict
from helpers import *

In [3]:
### Initialization of hard coded data ###

order_books = defaultdict(set)
df_ratio = pd.read_csv("ratio_dict.csv", index_col = 0).set_index("Pair")
df_global = pd.DataFrame({"date":[]})
df_num_adr_per_unit = {}
df_num_stock_per_unit = {}
for idx, row in df_ratio.iterrows():
    adr = idx.split("_")[0]
    stock = idx.split("_")[1]
    df_num_adr_per_unit[adr] = max(1, row["adr_num_per_stock"])
    df_num_stock_per_unit[stock] = max(1, row["stock_num_per_adr"])

aus_adr_underlying_pairs = [
    ["ATHE", "ATH"],["GENE", "GTG"],["IMMP", "IMM"],["IMRN","IMC"],["KZIA", "KZA"],
    ["MESO","MSB"],["PLL", "PLL"],["WBK","WBC"]
]

hk_adr_underlying_pairs = [
    ["ACH", "2600"],["BGNE", "6160"],["CEA","670"],["HNP", "902"],["LFC", "2628"],["PTR", "857"],
    ["SHI", "338"],["SNP","386"],["ZNH", "1055"]
]

jpn_adr_underlying_pairs = [
    ["CAJ", "7751"],["HMC","7267"],["IX","8591"],["MFG","8411"],["MUFG","8306"],["NMR","8604"],
    ["SMFG","8316"],["SONY","6758"],["TAK", "4502"],["TM", "7203"]
]

country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "HK":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}

forex_pairs = [("AUD", "USD"),  ("USD", "HKD"), ("USD","JPY")]


# Below are functions that assist in our date control

us_dst = {
        2021: (datetime(2021, 3, 14), datetime(2021, 11, 7)),
        2020: (datetime(2020, 3, 8),  datetime(2020, 11, 1)),
        2019: (datetime(2019, 3, 10), datetime(2019, 11, 3)),
        2018: (datetime(2018, 3, 11), datetime(2018, 11, 4)),
        2017: (datetime(2017, 3, 12), datetime(2017, 11, 5)),
        2016: (datetime(2016, 3, 13), datetime(2016, 11, 6)),
        2015: (datetime(2015, 3, 9),  datetime(2015, 11, 2)),
    } 

def next_weekday(date_now):
    if date_now.weekday()<4:
        return date_now + timedelta(days = 1)
    
    while date_now.weekday()>3:
        date_now += timedelta(days = 1)
        
    return date_now

def next_market_open_day(date_now):
    '''
    Returns today if Mon - Thurs, returns next Mon if Fri thru Sun
    '''
    while date_now.weekday()>4:
        date_now += timedelta(days = 1)
    return date_now

def AUS_opening(date_now):
    date_adj = next_market_open_day(date_now)
    us_start, us_end = us_dst[date_adj.year]
    if us_start < date_adj and date_adj < us_end:
        return datetime.combine(date_adj,time(10,0)) - timedelta(hours = 14)
    else:
        return datetime.combine(date_adj, time(10,0)) - timedelta(hours = 15)

def AUS_closing(date_now):
    open_time = AUS_opening(date_now)
    return open_time + timedelta(hours = 6)

def HK_opening(date_now):
    date_adj = next_market_open_day(date_now)
    start, end = us_dst[date_adj.year]
    if start < date_adj and date_adj < end:
        return datetime.combine(date_adj,time(9,30)) - timedelta(hours = 12)
    else:
        return datetime.combine(date_adj, time(9,30)) - timedelta(hours = 13)
    
def HK_closing(date_now):
    open_time = HK_opening(date_now)
    return open_time + timedelta(hours = 6.5)
    
def JPN_opening(date_now):
    date_now = next_market_open_day(date_now)
    start, end = us_dst[date_now.year]
    if start < date_now and date_now < end:
        return datetime.combine(date_now,time(9,0)) - timedelta(hours = 13)
    else:
        return datetime.combine(date_now,time(9,0)) - timedelta(hours = 14)
    
def JPN_closing(date_now):
    open_time = JPN_opening(date_now)
    return open_time + timedelta(hours = 6)

def US_opening(date_now):
    return datetime.combine(next_market_open_day(date_now),time(9,30))

def US_closing(date_now):
    return datetime.combine(next_market_open_day(date_now),time(16,0))

In [4]:
# initialize data lists, hyperparameters etc

path_data = "live_trading_data/"
df_forex = pd.DataFrame({"date":[]})
def initialize_forex():
    """
    Reads reads forex data from log to df_forex and if log is not updated, pull data until df_forex is updated then update log
    """
    
    global df_forex, country_info

    now = datetime.now()
    df_forex = pd.read_csv(f"{path_data}df_forex.csv")
    df_forex["date"] = [datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S') for date_time in df_forex["date"]]
    pull_head = datetime.combine(max(now-timedelta(days = round(8/5*100)),df_forex["date"].iloc[-1]).date(),time(17,0))


    now = datetime.combine(datetime.now().date(), time(17,5))
    df_temp_jpy = pd.DataFrame({"date":[]})
    df_temp_hkd = pd.DataFrame({"date":[]})
    df_temp_aud = pd.DataFrame({"date":[]})
    df_temp_holder = {
        "AUD.USD" : df_temp_aud,
        "JPY.USD" : df_temp_jpy,
        "HKD.USD" : df_temp_hkd
    }
    while pull_head < now:
        print(pull_head.date())
        for currency1, currency2 in forex_pairs:

            history = "2 D"
            freq = "1 min"
            side = "BID_ASK"

            forex_contract = Contract(symbol = currency1, secType = "CASH", exchange = "IDEALPRO", currency = currency2)
            ib.qualifyContracts(forex_contract)
            try:
                df_temp = get_data(forex_contract, history, freq, side, pull_head)    

                if currency2 == "USD":
                    df_temp[f"{currency1}.{currency2}"] = (df_temp["open"] + df_temp["close"])/2
                    forex_string = f"{currency1}.{currency2}"
                else:
                    df_temp[f"{currency2}.{currency1}"] = 2/(df_temp["open"] + df_temp["close"])
                    forex_string = f"{currency2}.{currency1}"

                df_temp["date"] = df_temp["date"].astype('datetime64[s]')
    #             if pull_head == datetime(2021,4,21): print(df_temp)

                df_temp_holder[forex_string] = pd.merge(df_temp_holder[forex_string], df_temp[["date",forex_string]], how = "outer")
            except Exception as e:
                print(e)

        pull_head += timedelta(days = 1)
    df_temp_2 = pd.merge(df_temp_holder["HKD.USD"],pd.merge(df_temp_holder["AUD.USD"], df_temp_holder["JPY.USD"],  how = "outer"), on = "date", how = "outer")
    df_forex = pd.merge(df_forex,df_temp_2, how = "outer").drop_duplicates(keep = "first")
    df_forex.to_csv(f"{path_data}df_forex.csv", index = False)
    print("df_forex initialized")

def save_forex():
    """
    Updates logged forex with current df_forex
    """
    global df_forex
    df_forex.drop_duplicates("date")
    df_forex.to_csv(f"{path_data}df_forex.csv", index = False)

In [5]:
country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "HK":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}


# Store all the historical adr/underlying open and close data

#define a function to get IB data. endDate is the last date of the historical data 

def get_data(contract, history, freq, side, endDate =''):
    """
    Gets historical data for specific contract for a given history at resolution freq 
    """
    bar = ib.reqHistoricalData(
        contract,
        endDateTime = endDate,
        durationStr=history,
        barSizeSetting= freq,
        whatToShow=side,
        useRTH=True,
        formatDate=1)
    return util.df(bar)

def pull_asian_close(days = 1):
    """
    Pull close data for all the asian markets for days number of days and append to df_global
    """
    global df_global, country_info

    today = datetime.now().date()
    history = f"{days} D"
    freq = "1 Day"
    side = 'Trades'

    df_asian_close = pd.DataFrame({"date":[]})
    for key in country_info.keys():
        pairs = country_info[key]["Pairs"]
        exchange = country_info[key]["Exchange"]
        currency = country_info[key]["Currency"]
        
        for p in pairs:
            underlying = p[1]
            contract = Contract(symbol = underlying, secType = 'STK', exchange = exchange, currency = currency)

            df = get_data(contract, history, freq, side, endDate =today)
            ratio = max(1, df_ratio.loc[f"{p[0]}_{p[1]}"]["stock_num_per_adr"])
            df[f"underlying_{underlying}_close_per_unit"] = df["close"] * ratio

            df = df.loc[:, ["date", "close", f"underlying_{underlying}_close_per_unit", "volume"]]
            col_name = "underlying_" + underlying + "_close"
            df = df.rename(columns = {"close":col_name, "volume": f"underlying_{underlying}_volume"})
            
            df_asian_close = pd.merge(df_asian_close, df, how = "outer")

            
    df_global = pd.merge(df_global, df_asian_close,how = "outer").sort_values("date")

    
    
def pull_us_close(days = 1):
    """
    Pull US close data for days number of days and append to df_global
    """
    global df_global, country_info

    today = datetime.now().date()
    history = f"{days} D"
    freq = "1 Day"
    side = 'Trades'
    
    df_us_close = pd.DataFrame({"date":[]})
    for key in country_info.keys():
        pairs = country_info[key]["Pairs"]
        exchange = "SMART"
        currency = "USD"
        
        for p in pairs:
            underlying = p[0] # get the ticker of the ADR
            contract = Contract(symbol = underlying, secType = 'STK', exchange = exchange, currency = currency)
            df = get_data(contract, history, freq, side, endDate ='')
            ratio = max(1, df_ratio.loc[f"{p[0]}_{p[1]}"]["adr_num_per_stock"])
            df[f"adr_{p[0]}_close_per_unit"] = df["close"] * ratio
            df = df.loc[:, ["date", "close", f"adr_{p[0]}_close_per_unit", "volume" ]]
            col_name = "adr_" + underlying + "_close"
            df["volume"] *= 100
            df = df.rename(columns = {"close":col_name, "volume": f"adr_{underlying}_volume"})
            df_us_close = pd.merge(df_us_close, df, how = 'outer')
            
    df_global = pd.merge(df_global, df_us_close,how = 'outer').sort_values("date")
    
def update_df_global_with_forex():
    """
    Appends forex at each respective open to df_global
    """
    global df_global, df_forex
    
    us_opens = [datetime.combine(x, time(9,30)) - timedelta(minutes = 30) for x in df_global["date"]]
    forex_before_us_opens = df_forex[df_forex["date"].isin(us_opens)].drop_duplicates(["date"]).sort_values(["date"]).reset_index(drop = True)
    forex_before_us_opens["date"] = [datetime.date(x) for x in forex_before_us_opens["date"]]
    forex_before_us_opens.columns = ["date", "JPY.USD_before_us_open", "AUD.USD_before_us_open", "HKD.USD_before_us_open"]

    # Add extra date
    dates = list(df_global["date"])
    last_date = dates[-1]
    dates.append(next_weekday(last_date))
    aus_opens = [AUS_opening(datetime.combine(x, time(0,0))) - timedelta(minutes = 30) for x in dates]
    hk_opens = [HK_opening(datetime.combine(x, time(0,0))) - timedelta(minutes = 30) for x in dates]
    jap_opens = [JPN_opening(datetime.combine(x, time(0,0))) - timedelta(minutes = 30) for x in dates]
    aus_forex_opens =  df_forex[df_forex["date"].isin(aus_opens)].drop_duplicates(["date"]).sort_values(["date"]).reset_index(drop = True)
    hk_forex_opens =  df_forex[df_forex["date"].isin(hk_opens)].drop_duplicates(["date"]).sort_values(["date"]).reset_index(drop = True)
    jap_forex_opens =  df_forex[df_forex["date"].isin(jap_opens)].drop_duplicates(["date"]).sort_values(["date"]).reset_index(drop = True)
    aus_forex_opens["date"] += timedelta(days = 1)
    hk_forex_opens["date"] += timedelta(days = 1)
    jap_forex_opens["date"] += timedelta(days = 1)
    aus_forex_opens["date"] = [datetime.date(x) for x in aus_forex_opens["date"]]
    hk_forex_opens["date"] = [datetime.date(x) for x in hk_forex_opens["date"]]
    jap_forex_opens["date"] = [datetime.date(x) for x in jap_forex_opens["date"]]
    forex_before_asian_opens = pd.merge(pd.merge(aus_forex_opens[["date", "AUD.USD"]], jap_forex_opens[["date", "JPY.USD"]], how = "outer"),
                                        hk_forex_opens[["date", "HKD.USD"]], how = "outer")
    forex_before_asian_opens.columns = ["date", "AUD.USD_before_asian_open", "JPY.USD_before_asian_open", "HKD.USD_before_asian_open"]

    df_global = pd.merge(df_global, 
                         pd.merge(forex_before_us_opens, forex_before_asian_opens, how = "outer"),
                         how = "outer").sort_values("date")
    df_global = df_global[df_global["date"] != date(2020, 12, 25)]
    df_global.reset_index(inplace = True, drop = True)

def pull_forex_data(history = "2 D", freq = "1 min", side = "BID_ASK", end_date = "" ):
    '''
    Updates all three forex pairs
    '''
    
    global df_global, country_info, forex_pairs, test, df_forex

    df_temp = pd.DataFrame({"date":[]})
    for currency1, currency2 in forex_pairs:
        contract = Contract(symbol = currency1, secType = 'CASH', exchange = "IDEALPRO", currency = currency2)
        df = get_data(contract, history, freq, side)

        if currency2 == "USD":
            df[f"{currency1}.{currency2}"] = (df["open"] + df["close"])/2
            forex_string = f"{currency1}.{currency2}"
        else:
            df[f"{currency2}.{currency1}"] = 2/(df["open"] + df["close"])
            forex_string = f"{currency2}.{currency1}"
        df_temp = pd.merge(df_temp, df[['date',forex_string]], how = "outer")
    
    df_forex = pd.merge(df_forex, df_temp, how = "outer")
    df_forex.drop_duplicates(inplace = True)
    update_df_global_with_forex()
    
def convert_price_asian_close():
    
    """
    Convert the last Asian prices we pulled before US open into US dollars
    Using the most recent forex data
    """
    global df_global, df_forex, df_ratio, country_info
    
    # We use the last 
    for key in country_info:
        pairs = country_info[key]["Pairs"]
        for pair in pairs:
            underlying_col_name = f"underlying_{pair[1]}_close"
            pair_name =  pair[0] + "_" + pair[1]
            ratio = df_ratio.loc[pair_name,"adr_num_per_stock"] # Determing number of adr per stock
            
            forex_col_name = country_info[key]["Currency"]+".USD"
            fx = df_forex.loc[len(df_forex)-1, forex_col_name]
            price_in_usd = df_global.loc[len(df_global)-1, underlying_col_name]*ratio*fx
            
            df_global.at[len(df_global)-1, underlying_col_name] = price_in_usd

In [6]:
def check_open_order(security, sectype = "underlying"):
    """
    Check if we have open order for this particular security
    Args: 
    security (str) name of security
    sectype(str): takes th evalue either "adr" or "underlying"
    
    return: the quantity of open orders (orders that we wanted to trade but not traded) of a particular security
    
    """
    global order_books
    ct = 0
    lookup = security +"_"+ sectype
    all_orders = order_books[lookup]
    open_orders = ib.openOrders()
    for o in openOders:
        if o.orderId in all_orders:
            ct += o.totalQuantity
        
    return ct

In [7]:
# Pairs we have chosen to trade and trading limits
trading_limits = {"Australia" : 2, "China" : 2, "Japan" : 2}
allocation = {"Australia" : 0.25, "China" : 0.4, "Japan" : 0.35}
country_to_forex = {"Australia" : "AUD.USD", "China" : "HKD.USD", "Japan" : "JPY.USD"}

list_pairs = [("Japan", "SMFG_8316"), 
              ("Japan", "IX_8591"),
              ("Japan", "TM_7203"),
              ("Japan", "MFG_8411"),
              ("China", "BGNE_6160"), 
              ("China", "SNP_386"), 
              ("China", "HNP_902"), 
              ("China", "CEA_670"),
              ("China", "ACH_2600"),
              ("Australia", "MESO_MSB"),
              ("Australia", "IMMP_IMM"),
              ("Australia", "PLL_PLL"),
              ("Australia", "KZIA_KZA"),
              ("Australia", "IMRN_IMC")]

full_list_pairs = [('Australia', 'ATHE_ATH'),
                   ('Australia', 'GENE_GTG'),
                   ('Australia', 'IMMP_IMM'),
                   ('Australia', 'IMRN_IMC'),
                   ('Australia', 'JHX_JHX'),
                   ('Australia', 'KZIA_KZA'),
                   ('Australia', 'MESO_MSB'),
                   ('Australia', 'PLL_PLL'),
                   ('Australia', 'WBK_WBC'),
                   ('China', 'ACH_2600'),
                   ('China', 'BGNE_6160'),
                   ('China', 'CEA_670'),
                   ('China', 'HNP_902'),
                   ('China', 'LFC_2628'),
                   ('China', 'PTR_857'),
                   ('China', 'SHI_338'),
                   ('China', 'SNP_386'),
                   ('China', 'ZNH_1055'),
                   ('Japan', 'CAJ_7751'),
                   ('Japan', 'HMC_7267'),
                   ('Japan', 'IX_8591'),
                   ('Japan', 'MFG_8411'),
                   ('Japan', 'MUFG_8306'),
                   ('Japan', 'NMR_8604'),
                   ('Japan', 'SMFG_8316'),
                   ('Japan', 'SONY_6758'),
                   ('Japan', 'TAK_4502'),
                   ('Japan', 'TM_7203')]

## Intialize Hyperparameter
fname = 'logs/results_all.txt'
with open(fname, 'r') as f:
    is_res = f.readlines()
hp = {}
for i in range(len(full_list_pairs)):
    hp[full_list_pairs[i]] = [float(x) for x in is_res[i*5 + 4].split("(")[1].split(")")[0].split(", ")]
    
hp_dict = {}
for (country, adr) in list_pairs:
    hp_dict[(country, adr)] = {}
    hp_dict[(country, adr)]["lookback"] = int(hp[(country, adr)][0])
    hp_dict[(country, adr)]["entry"] = hp[(country, adr)][1]
    hp_dict[(country, adr)]["exit"] = hp[(country, adr)][2]
    hp_dict[(country, adr)]["stop_loss"] = hp[(country, adr)][3]
    # Fraction of cash allocated to each adr-stock pair
    hp_dict[(country, adr)]["allocation"] = allocation[country]/trading_limits[country]
    hp_dict[(country, adr)]["original_allocation"] = allocation[country]/trading_limits[country]
    
actual_hps = pd.read_csv("actual_hps.csv", index_col = 0)
for idx, row in actual_hps.iterrows():
    country = row["country"]
    adr = row["adr"]
    hp_dict[(country, adr)]["limit_exit"] = max(row["actual_exit"], hp_dict[(country, adr)]["exit"])
    hp_dict[(country, adr)]["limit_entry"] = min(row["actual_entry"], hp_dict[(country, adr)]["entry"])
    if hp_dict[(country, adr)]["limit_entry"] < hp_dict[(country, adr)]["limit_exit"]:
        hp_dict[(country, adr)]["limit_entry"] = hp_dict[(country, adr)]["entry"]
    
## initialize diff_record_dict
diff_record_dict = {}
for (country, adr) in list_pairs:
    diff_record_dict[(country, adr)] = deque(maxlen = 2*hp_dict[(country, adr)]["lookback"])
    
## initialize entry condtitions for each adr pair
conditions = {}
for (country, adr) in list_pairs:
    conditions[(country, adr)] = {}
    conditions[(country, adr)]["enter_cond1"] = False
    conditions[(country, adr)]["exit_cond1"] = False
    conditions[(country, adr)]["enter_cond2"] = False
    conditions[(country, adr)]["exit_cond2"] = False

## initialize positions dictionary for each pair
positions = {}
for (country, adr) in list_pairs:
    positions[(country, adr)] = {}
    positions[(country, adr)]["stock_pos"] = 0
    positions[(country, adr)]["adr_pos"] = 0
    positions[(country, adr)]["prev_adr_pos"] = 0
    positions[(country, adr)]["holding_period"] = None
    positions[(country, adr)]["trade_type"] = None

## initialize actions to take
actions_to_take = {}
for (country, adr) in list_pairs:
    actions_to_take[(country, adr)] = {}

## intialize risk and portfolio metrics
cash = 249000
var_limit = 0.1
max_drawdown_limit = 0.2
sigma_limit = 0.05
stop_loss_limit = 0.15

In [8]:
def bef_aus_jap_open():
    """
    Check if we are currently before ASX/TSEJ open
    """
    return datetime.now().time() >= time(19, 0) and datetime.now().time() <= time(20, 45)
    #return True
#     return False

def bef_hk_open():
    """
    Check if we are currently before SEHK open
    """
    return datetime.now().time() > time(20, 45) and datetime.now().time() <= time(22, 30)
    # return True
#     return False
    
def bef_us_open():
#     return True
    """
    Check if we are currently before NYSE open
    """
    return datetime.now().time() >= time(8, 30) and datetime.now().time() <= time(10, 30)

In [9]:
def populate_diff_queue():
    """
    Using the data from df_global, we append latest adjust price difference values to the queue
    """
    global list_pairs, diff_record_dict, df_global
    for country, adr in list_pairs:
        ticker_ADR, ticker_underlying = adr.split("_")
        diff_record = deque(maxlen = 2*hp_dict[(country, adr)]["lookback"])
        for idx, row in df_global.iterrows():
            if bef_us_open():
                if idx > 0:
                    prev_row = df_global.loc[idx - 1]
                    # Append Asian Open data
                    adr_price = prev_row[f"adr_{ticker_ADR}_close_per_unit"]
                    stock_price = prev_row[f"underlying_{ticker_underlying}_close_per_unit"]
                    adjusted_stock_price = stock_price*row[f"{country_to_forex[country]}_before_asian_open"]
                    diff = adr_price - adjusted_stock_price
                    if not np.isnan(diff):
                        diff_record.append(diff)

                    # Append US Open data
                    adr_price = prev_row[f"adr_{ticker_ADR}_close_per_unit"]
                    stock_price = row[f"underlying_{ticker_underlying}_close_per_unit"]
                    adjusted_stock_price = stock_price*row[f"{country_to_forex[country]}_before_us_open"]
                    diff = adr_price - adjusted_stock_price
                    if not np.isnan(diff):
                        diff_record.append(diff)

            elif bef_aus_jap_open() or bef_hk_open():
                if idx > 0 and idx < len(df_global) - 1:
                    prev_row = df_global.loc[idx - 1]
                    next_row = df_global.loc[idx + 1]

                    # Append US Open data
                    adr_price = prev_row[f"adr_{ticker_ADR}_close_per_unit"]
                    stock_price = row[f"underlying_{ticker_underlying}_close_per_unit"]
                    adjusted_stock_price = stock_price*row[f"{country_to_forex[country]}_before_us_open"]
                    diff = adr_price - adjusted_stock_price
                    if not np.isnan(diff):
                        diff_record.append(diff)

                    # Append Asian Open data
                    adr_price = row[f"adr_{ticker_ADR}_close_per_unit"]
                    stock_price = row[f"underlying_{ticker_underlying}_close_per_unit"]
                    adjusted_stock_price = stock_price*next_row[f"{country_to_forex[country]}_before_asian_open"]
                    diff = adr_price - adjusted_stock_price
                    if not np.isnan(diff):
                        diff_record.append(diff)

        diff_record_dict[(country,adr)] = diff_record

In [10]:
def calculate_z_score(diff_record):
    """
    Calculates z score of a price adjusted price difference series and returns means standard deviation and z score
    """
    mean = np.array(diff_record).mean()
    std = np.array(diff_record).std()
    return mean, std, (diff_record[-1] - mean)/std

In [11]:
def populate_actions_to_take():
    """
    Calculates actions to take based on df_global and updates the fields of actions_to_take
    """
    
    global list_pairs, diff_record_dict, df_global, actions_to_take
    
    if bef_aus_jap_open():
        pairs_to_evaluate = [x for x in list_pairs if x[0] in ["Australia", "Japan"]]
    elif bef_hk_open():
        pairs_to_evaluate = [x for x in list_pairs if x[0] == "China"]
    elif bef_us_open():
        pairs_to_evaluate = list_pairs
    else:
        pairs_to_evaluate = []
        
    for (country, adr) in pairs_to_evaluate:
        ticker_ADR, ticker_underlying = tuple(adr.split("_"))
        
#         if len(actions_to_take[(country, adr)]) == 1:
#             for key in actions_to_take[(country, adr)]:
#                 # Update volume
#                 actions_to_take[(country, adr)][key]["volume"] = "MATCH POSITION"
                
#         elif len(actions_to_take[(country, adr)]) == 0:
        # Calculate
        mean, std, z_score = calculate_z_score(diff_record_dict[(country,adr)])
        entry = hp_dict[(country, adr)]["entry"]
        exit = hp_dict[(country, adr)]["exit"]
        print(z_score, entry)
        ### MISSING NO POSITION COMMENT
        if z_score > entry:
            # Enter
            # Market orders
            if bef_aus_jap_open() or bef_hk_open():
                # Calculate volume and price
                # Before Asian open
                stock_adt = df_global.loc[len(df_global)-6:len(df_global)-2][f"underlying_{ticker_underlying}_volume"].dropna().median()
                adr_adt = df_global.loc[len(df_global)-6:len(df_global)-2][f"adr_{ticker_ADR}_volume"].dropna().median()
                adr_volume = (0.1*adr_adt)/df_num_adr_per_unit[ticker_ADR]
                stock_volume = (0.1*stock_adt)/df_num_stock_per_unit[ticker_underlying]
                adr_price_per_unit = df_global.loc[len(df_global)-2, f"adr_{ticker_ADR}_close_per_unit"]
                stock_price_per_unit = df_global.loc[len(df_global)-2, f"underlying_{ticker_underlying}_close_per_unit"]*df_global.loc[len(df_global)-1, f"{country_to_forex[country]}_before_asian_open"]
                print(df_global.loc[len(df_global)-1, f"{country_to_forex[country]}_before_asian_open"])
                units = int(min((hp_dict[(country, adr)]["allocation"]*cash)/adr_price_per_unit,
                                (hp_dict[(country, adr)]["allocation"]*cash)/stock_price_per_unit, 
                                adr_volume, 
                                stock_volume))
                adr_quantity = int(units*df_num_adr_per_unit[ticker_ADR])
                stock_quantity = int(units*df_num_stock_per_unit[ticker_underlying])
                stock_values = np.array(df_global.loc[len(df_global)-101:len(df_global)-2, f"underlying_{ticker_underlying}_close"]*stock_quantity)
                adjusted_stock_values = stock_values*np.array(df_global.loc[len(df_global)-100:len(df_global)-1, f"{country_to_forex[country]}_before_asian_open"])
                adr_values = np.array(df_global.loc[len(df_global)-101:len(df_global)-2, f"adr_{ticker_ADR}_close"]*adr_quantity)
                mask = np.isnan(adjusted_stock_values) | np.isnan(adr_values)
                adjusted_stock_values = adjusted_stock_values[~mask]
                adr_values = adr_values[~mask]
                sigma, var, max_drawdown_abs = get_risk_statistics(adjusted_stock_values, adr_values, var_ci = 0.95)
                adjusted_cash = cash*allocation[country]
                if (var > adjusted_cash*var_limit or 
                    max_drawdown_abs > max_drawdown_limit*adjusted_cash or 
                    sigma > adjusted_cash*sigma_limit):
                    frac = min((adjusted_cash*var_limit)/var, 
                               (adjusted_cash*max_drawdown_limit)/max_drawdown_abs,
                              (adjusted_cash*sigma_limit)/sigma)
                    units = int(frac*units)
                    if units == 0:
                        continue
                    adr_quantity = int(units*df_num_adr_per_unit[ticker_ADR])
                    stock_quantity = int(units*df_num_stock_per_unit[ticker_underlying])

                execution_stock_price_per_unit = adr_price_per_unit - (mean + hp_dict[(country, adr)]["limit_entry"]*std)
                execution_stock_price = execution_stock_price_per_unit/df_num_stock_per_unit[ticker_underlying]
                execution_stock_price_in_local_currency = execution_stock_price/df_global.loc[len(df_global)-1, f"{country_to_forex[country]}_before_asian_open"]
                liquidation_stock_price_per_unit = adr_price_per_unit - (mean + hp_dict[(country, adr)]["limit_exit"]*std)
                liquidation_stock_price = liquidation_stock_price_per_unit/df_num_stock_per_unit[ticker_underlying]
                liquidation_stock_price_in_local_currency = liquidation_stock_price/df_global.loc[len(df_global)-1, f"{country_to_forex[country]}_before_asian_open"]

                actions_to_take[(country, adr)]["stock"] = {}
                actions_to_take[(country, adr)]["stock"]["action"] = "buy"
                actions_to_take[(country, adr)]["adr"] = {}
                actions_to_take[(country, adr)]["adr"]["action"] = "sell"
                actions_to_take[(country, adr)]["stock"]["volume"] = stock_quantity
                actions_to_take[(country, adr)]["stock"]["execution_price"] = execution_stock_price_in_local_currency
                actions_to_take[(country, adr)]["stock"]["liquidation_price"] = liquidation_stock_price_in_local_currency
                actions_to_take[(country, adr)]["adr"]["volume"] = -1
                actions_to_take[(country, adr)]["adr"]["execution_price"] = -1
            else: 
                # Calculate volume and price
                # Before US open
                stock_adt = df_global.loc[len(df_global)-5:len(df_global)-1][f"underlying_{ticker_underlying}_volume"].dropna().median()
                adr_adt = df_global.loc[len(df_global)-6:len(df_global)-2][f"adr_{ticker_ADR}_volume"].dropna().median()
                adr_volume = (0.1*adr_adt)/df_num_adr_per_unit[ticker_ADR]
                stock_volume = (0.1*stock_adt)/df_num_stock_per_unit[ticker_underlying]
                adr_price_per_unit = df_global.loc[len(df_global)-2, f"adr_{ticker_ADR}_close_per_unit"]
                stock_price_per_unit = df_global.loc[len(df_global)-1, f"underlying_{ticker_underlying}_close_per_unit"]*df_global.loc[len(df_global)-1, f"{country_to_forex[country]}_before_us_open"]
                units = int(min((hp_dict[(country, adr)]["allocation"]*cash)/adr_price_per_unit,
                                (hp_dict[(country, adr)]["allocation"]*cash)/stock_price_per_unit, 
                                adr_volume, 
                                stock_volume))
                adr_quantity = int(units*df_num_adr_per_unit[ticker_ADR])
                stock_quantity = int(units*df_num_stock_per_unit[ticker_underlying])
                stock_values = np.array(df_global.loc[len(df_global)-100:len(df_global)-1, f"underlying_{ticker_underlying}_close"]*stock_quantity)
                adjusted_stock_values = stock_values*np.array(df_global.loc[len(df_global)-100:len(df_global)-1, f"{country_to_forex[country]}_before_us_open"])
                adr_values = np.array(df_global.loc[len(df_global)-101:len(df_global)-2, f"adr_{ticker_ADR}_close"]*adr_quantity)
                mask = np.isnan(adjusted_stock_values) | np.isnan(adr_values)
                adjusted_stock_values = adjusted_stock_values[~mask]
                adr_values = adr_values[~mask]
                sigma, var, max_drawdown_abs = get_risk_statistics(adjusted_stock_values, adr_values, var_ci = 0.95)
                adjusted_cash = cash*allocation[country]
                if (var > adjusted_cash*var_limit or 
                    max_drawdown_abs > max_drawdown_limit*adjusted_cash or 
                    sigma > adjusted_cash*sigma_limit):
                    frac = min((adjusted_cash*var_limit)/var, 
                               (adjusted_cash*max_drawdown_limit)/max_drawdown_abs,
                              (adjusted_cash*sigma_limit)/sigma)
                    print(frac)
                    units = int(frac*units)
                    if units == 0:
                        continue
                    adr_quantity = int(units*df_num_adr_per_unit[ticker_ADR])
                    stock_quantity = int(units*df_num_stock_per_unit[ticker_underlying])

                execution_adr_price_per_unit = stock_price_per_unit + mean + hp_dict[(country, adr)]["limit_entry"]*std
                execution_adr_price = execution_adr_price_per_unit/df_num_adr_per_unit[ticker_ADR]
                liquidation_adr_price_per_unit = stock_price_per_unit + mean + hp_dict[(country, adr)]["limit_exit"]*std
                liquidation_adr_price = liquidation_adr_price_per_unit/df_num_adr_per_unit[ticker_ADR]

                actions_to_take[(country, adr)]["stock"] = {}
                actions_to_take[(country, adr)]["stock"]["action"] = "buy"
                actions_to_take[(country, adr)]["adr"] = {}
                actions_to_take[(country, adr)]["adr"]["action"] = "sell"
                actions_to_take[(country, adr)]["adr"]["volume"] = adr_quantity
                actions_to_take[(country, adr)]["adr"]["execution_price"] = execution_adr_price
                actions_to_take[(country, adr)]["adr"]["liquidation_price"] = liquidation_adr_price
                actions_to_take[(country, adr)]["stock"]["volume"] = -1
                actions_to_take[(country, adr)]["stock"]["execution_price"] = -1                
            ### MISSING POSITION CHECK
        elif z_score < exit:
            actions_to_take[(country, adr)]["stock"] = {}
            actions_to_take[(country, adr)]["stock"]["action"] = "sell"
            # Update volume
            actions_to_take[(country, adr)]["stock"]["volume"] = "LIQUIDATE"
            actions_to_take[(country, adr)]["stock"]["execution_price"] = -1
            actions_to_take[(country, adr)]["adr"] = {}
            actions_to_take[(country, adr)]["adr"]["action"] = "buy"
            # Update volume
            actions_to_take[(country, adr)]["adr"]["volume"] = "LIQUIDATE"
            actions_to_take[(country, adr)]["adr"]["execution_price"] = -1

In [12]:
def execute_actions(active_country):
    """
    Iterates through actions to take and places orders and liquidation orders as necessary
    """
    global list_pairs, diff_record_dict, df_global, actions_to_take
    if active_country == "US":
        for country, adr in list_pairs:
            ticker_adr, ticker_stk = adr.split("_")
            
            if len(actions_to_take[(country, adr)]) == 1:
                adr_action = actions_to_take[(country, adr)]["adr"]
                if adr_action["volume"] == "LIQUIDATE":
                    volume = portfolio[(country, adr)][ticker_adr+"_adr"]
                    # liquidate existing position
                    place_order(country, ticker_adr, "adr", adr_action["action"], volume, -1, market_order = True)
                else:
                    volume = portfolio[(country, adr)][ticker_stk + "_stk"]*df_ratio[adr]["stock_num_per_adr"]
                    # complete pair
                    place_order(country, ticker_adr, "adr", adr_action["action"], volume, -1, market_order = True)
                    # early liquidation order
                    place_order(country, ticker_adr, "adr", "buy", volume, adr_action["liquidation_price"])
                actions_to_take[(country,adr)].pop("adr")
            elif len(actions_to_take[(country,adr)]) == 2:
                adr_action = actions_to_take[(country, adr)]["adr"]
                # entet pair
                place_order(country, ticker_adr, "adr", adr_action["action"], adr_action["volume"], adr_action["execution_price"])
                # early liquidation order
                place_order(country, ticker_adr, "adr", "buy", adr_action["volume"], adr_action["liquidation_price"])
                actions_to_take[(country,adr)].pop("adr")
        return
    
    # any other country besides US
    for (country, adr) in list_pairs:
        if country != active_country: # only consider actions for stocks taken whose market is opening
            continue
        
        ticker_adr, ticker_stk = adr.split("_")
        if len(actions_to_take[(country, adr)]) == 1:
            stock_action = actions_to_take[(country, adr)]["stock"]
            if adr_action["volume"] == "LIQUIDATE":
                volume = portfolio[(country, adr)][ticker_adr+"_stk"]
                # liquidate existing position
                place_order(country, ticker_stk, "stk", stock_action["action"], volume, -1, market_order = True)
            else:
                volume = portfolio[(country, adr)][ticker_adr + "_adr"]*df_ratio[adr]["adr_num_per_stock"]
                # complete pair
                place_order(country, ticker_stk, "stk", stock_action["action"], volume, stock_action["execution_price"])
                # early liquidation order
                place_order(country, ticker_stk, "stk", "sell", volume, stk_action["liquidation_price"])
            actions_to_take[(country,adr)].pop("stock")
        elif len(actions_to_take[(country,adr)]) == 2:
            # enter pair
            place_order(country, ticker_stk, "stk", stock_action["action"], stock_action["volume"], stock_action["execution_price"])
            # early liquidation order
            place_order(country, ticker_stk, "stk", "buy", stock_action["volume"], stock_action["liquidation_price"])
            actions_to_take[(country,adr)].pop("stock")

In [13]:
def place_order(country, security, sectype, side, quantity, price, market_order = False):
    """
    Places a market or limit order and keeps track of order and position in portfolio
    """
    global order_books
    exchange = country_info[country]["Ex change"]
    currenct = country_info[country]["Currency"]
    
    # Define order 
    contract = Contract(symbol = security, secType = 'STK', exchange =exchange, currency = currency)
    if market_order == True:
        order =  MarketOrder(side,quantity)
    else:
        order = LimitOrder(side,quantity,price)
    
    # Place the order
    msg = ib.placeOrder(contract, order)
    orderId = msg.order.orderId
    
    # Add the orderId to orderbook
    name =  security +"_"+ sectype
    order_books[name].add(orderId)
    order_records[orderId] = my_order(security, side, sectype, quantity)
    

def print_order_records():
    """
    Prints order record into terminal
    """
    global order_records
    
    #sys.__stdout__.write("orderId\tSecurity\tSecType\tOrderType\tStatus\tTotal\tFilled\tAvgPrice\n")
    print(datetime.now())
    print("orderId\tSecurity\tSecType\tOrderType\tStatus\tTotal\tFilled\tAvgPrice\n")
    for order in order_records:
        orderId = order
        Security = order_records[order].sec_name
        Type = order_records[order].sec_type
        Status = order_records[order].status
        Total = order_records[order].total_quantity
        Filled = order_records[order].filled_quantity
        AvgPrice = order_records[order].avg_filled_price
        order_type = order_records[order].order_type
        sys.__stdout__.write(f"{orderId}\t{Security}\t\t{Type}\t{order_type}\t\t{Status}\t{Total}\t{Filled}\t{AvgPrice}\n")


def print_portfolio():
    """
    Prints current portfolio into notebook
    """
    global portfolio
    #sys.__stdout__.write("Security\tPosition\n")
    print(datetime.now())
    print("Security\tPosition\n")
    for key in portfolio:
        
        if len(key) <= 7:
            #sys.__stdout__.write(f"{key}\t\t{portfolio[key]}\n")
            print(f"{key}\t\t{portfolio[key]}\n")
        else:
            print(f"{key}\t{portfolio[key]}\n")
            #sys.__stdout__.write(f"{key}\t{portfolio[key]}\n")

In [14]:
## Initialize portfolio dictionary which stores current state of portfolio

portfolio = {"USD":0, "HKD": 0, "AUD": 0, "JPY": 0}
for key in country_info:
    for p in country_info[key]["Pairs"]:
        adr = p[0]+"_adr"
        portfolio[adr] = 0
        
        under = p[1] + "_stk"
        portfolio[under] = 0
        
        

class my_order:
    """
    Struct to hold order status
    """
    def __init__(self, sec_name, order_type, sec_type, total_quantity):
        
        self.sec_name = sec_name
        self.sec_type = sec_type # if foreign currency, put ""
        self.order_type = order_type # Either "buy" or "sell"
        self.status = "Pending"
        self.total_quantity = total_quantity

        self.filled_quantity = 0
        self.avg_filled_price = 0.0

def reset_portfolio():
    """
    Resets portfolio
    """
    global portfolio
        
    for key in country_info:
        for p in country_info[key]["Pairs"]:
            adr = p[0]+"_adr"
            portfolio[adr] = 0

            under = p[1] + "_stk"
            portfolio[under] = 0

        

def onOrderStatus(trade): 
    """
    Check order status for each trade made
    """
    global order_records, portfolio
    print(f"Update in order {trade.order.orderId}")
#     print(trade.orderStatus.status == 'Cancelled')
    #print('order status' + trade.orderStatus.status)
    orderId = trade.order.orderId
    if trade.orderStatus.status == 'Cancelled':
        order_records[orderId].status = "Cancelled"
        print("Order has been cancelled.")
    elif trade.orderStatus.status == "Filled":
        
        change = trade.orderStatus.filled - order_records[orderId].filled_quantity
        order_records[orderId].filled_quantity = trade.orderStatus.filled
        order_records[orderId].avg_filled_price = trade.orderStatus.avgFillPrice
        
        sec_type = order_records[orderId].sec_type
        sec =  order_records[orderId].sec_name
        order_type = order_records[orderId].order_type
        if sec_type == 'adr':
            if order_records[orderId].order_type == "buy":
                portfolio[sec + sec_type] += change
            else:
                portfolio[sec + sec_type] -= change
                
        elif sec_type == "underlying":
            if order_records[orderId].order_type == "buy":
                portfolio[sec + '_stk'] += change
            else:
                portfolio[sec +  '_stk'] -= change
        
        
        print(f"{trade.orderStatus.filled} shares of {order_type} {sec} ({sec_type}) has been filled at an average price of {trade.orderStatus.avgFillPrice}")
        if rade.orderStatus.remaining == 0:
            order_records[orderId].status = "Completed"
            print("Order is completed!")
            
    else:
        print(trade.orderStatus.status)
    

def liquidate():
    """
    Func to liquidate all our current positions
    
    """
    
    global portfolio
    positions = ib.positions()  # A list of positions, according to IB
    for position in positions:
        action = ""
        contract = position.contract
        if position.position > 0: # Number of active Long positions
            action = 'Sell' # to offset the long positions
        elif position.position < 0: # Number of active Short positions
            action = 'Buy' # to offset the short positions
        else:
            assert False
        totalQuantity = abs(position.position)
        order = MarketOrder(action=action, totalQuantity=totalQuantity)
        trade = ib.placeOrder(contract, order)
        orderId = trade.orderId
        security = trade.contract.symbol
        sectype = ""
        if trade.contract.currency == "USD": # if usd, we know it is an adr
            sectype = "adr"
        else:
            sectype = "stk"
            

            # Add the orderId to orderbook
        name =  security +"_"+ sectype
        order_books[name].add(orderId)
        order_records[orderId] = my_order(security, action, sectype, position)


        print(f'Flatten Position: {action} {totalQuantity} {contract.localSymbol}')
    reset_portfolio()
    

def onExecDetails(trade,fill): 
    """
    Test function for order placing
    """
    print('order exec details' + fill.execution)
    ib.execDetailsEvent+=onExecDetails 
    ib.orderStatusEvent+=onOrderStatus

    # fut = Future('ES', '20181221', 'GLOBEX') 
    # ib.qualifyContracts(fut) 

    contract = Stock('TSLA', 'SMART', 'USD')
    ib.qualifyContracts(contract)


    order =LimitOrder('BUY', 1,1) 
    trade = ib.placeOrder(contract, order) 
    print(trade)
    order_records[trade.order.orderId] = my_order("AAPL","Stock","buy",1)

    ib.run()

In [15]:
# 8.30 am
    # Pull Historical Forex Data (Minute, for lookback window)
    # Pull Close Data for Asian Market at 9am (Before US Market opens)
# 9.28 am 
    # Pull Forex Data for three currencies (For Z score)
    # Calculate Z Score 
    # for each pair:
        # if invested:
            # if risk condition or exit condition:
                # liquidate
            # else:
                # continue
        # if entry condition:
            # determine order size from risk metric and ADT over last 5 days
            # if 0: no order made
            # else: 
                # Define contract
                # make order (What order? Market Order?)


# 6 pm
    # Pull Close Data for US Market at 6pm (Before Asian Market opens)
    # Pull Historical Forex Data

# 2 min before Asian Market Open
    # Pull Forex Data
    # Calculate Z score using market close and minute forex data

In [16]:
country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "HK":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}
  
def intraday_trader(active_country, live_data):
    return

def check_pending_orders(active_country, live_data):
    return
    
def pull_orders_data():
    return

live_data = {}
def pull_live_data():
    """
    Pulls limit tick data for intraday trading
    """
    global live_data, country_info, forex_pairs
    
    dict_forex = {}
    for country in forex_pairs:
        currency1, currency2, not_used = country

        history = "60 S"
        freq = "1 min"
        side = "BID_ASK"

        forex_contract = Contract(symbol = currency1, secType = "CASH", exchange = "IDEALPRO", currency = currency2)
        ib.qualifyContracts(forex_contract)
        df_temp = get_data(forex_contract, history, freq, side)
        
        latest_tick = df_temp.iloc[-1]
        if currency2 == "USD":
            live_data[f"{currency1}.{currency2}"] = (latest_tick["open"] + latest_tick["close"])/2

        else:
            live_data[f"{currency2}.{currency2}"] = 2/(latest_tick["open"] + latest_tick["close"])
            
    for country in country_info.keys():
        currency = country_info[country]["Currency"]
        exchange = country_info[country]["Exchange"]
        pairs_list = country_info[country]["Pairs"]
        
        side = "Trades"

        for ticker_ADR, ticker_underlying in pairs_list:
            contract = Contract(symbol = ticker_ADR, secType = "STK", exchange = "SMART", currency = currency)
            ib.qualifyContracts(contract)
            df_temp = get_data(contract, history, freq, side)
            latest_tick = df_temp.iloc[-1]
            live_data[ticker_ADR] = latest_tick['close'] * max(df_ratio[(country, ticker_ADR + "_" + ticker_underlying)]["adr_num_per_stock"], 1)
            
            
            contract = Contract(symbol = ticker_underlying, secType = "STK", exchange = exchange, currency = currency)
            ib.qualifyContracts(contract)
            df_temp = get_data(contract, history, freq, side)
            latest_tick = df_temp.iloc[-1]
            ratio = max(df_ratio[(country, ticker_ADR + "_" + ticker_underlying)]["stock_num_per_adr"], 1)
            if country == "Australia":
                live_data[ticker_underlying] = latest_tick['close'] * live_data["AUD.USD"] * ratio
            elif country == "Japan":
                live_data[ticker_underlying] = latest_tick['close'] * live_data["JPY.USD"] * ratio
            elif country == "HK":
                live_data[ticker_underlying] = latest_tick['close'] * live_data["HKD.USD"] * ratio


In [ ]:
initialize_forex()
pull_asian_close(120)
pull_us_close(120)
pull_forex_data()

for (country, adr) in list_pairs:
    actions_to_take[(country, adr)] = {}
diff_record_dict = {}
for (country, adr) in list_pairs:
    diff_record_dict[(country, adr)] = deque(maxlen = 2*hp_dict[(country, adr)]["lookback"])

# update date
while True:
    # keep track of what day it is
    today = datetime.now() # ideally the algorithm starts at 8am in the morning
    
    # keep track of how many times we pull data for calibration
    bef_us_pull = False
    bef_jpn_pull = False
    bef_aus_pull = False
    bef_hk_pull = False
    
    # Various Open and Close hours for various markets
    
    US_open = US_opening(today)
    US_close = US_closing(today)
    HK_open = HK_opening(today + timedelta(days = 1))
    HK_close = HK_closing(today + timedelta(days = 1))
    JPN_open = JPN_opening(today + timedelta(days = 1))
    JPN_close = JPN_closing(today + timedelta(days = 1))
    AUS_open = AUS_opening(today + timedelta(days = 1))
    AUS_close = AUS_closing(today + timedelta(days = 1))
    
    now = datetime.now()
    
    while now < US_close or now < HK_close or now < JPN_close or now < AUS_close:
        print(now)
        '''
        US Market
        '''
        # Before US Market Open pull historical data
        if not bef_us_pull and now > US_open - timedelta(minutes = 30):
            bef_us_pull = True
            pull_asian_close(2)
            pull_us_close(2)
            pull_forex_data()
            populate_diff_queue()
            populate_actions_to_take()
            execute_actions("US")

        if US_open < now and now < US_close:
            # Pull minute data to check whether to liquidate positions
            live_data = pull_live_data()
            intraday_trader("US", live_data)
            
        if US_close - timedelta(minutes = 10) < now and now < US_close:
            # check pending limit orders and liquidate
            live_data = pull_live_data()
            check_pending_orders("US", live_data)
                        
        '''
        Asian Markets
        '''
        
        # before asian markets open, pull us market close
        if not bef_hk_pull and now > HK_open - timedelta(minutes = 30):
            bef_hk_pull = True
            pull_asian_close(2)
            pull_us_close(2)
            pull_forex_data()
            populate_diff_queue()
            populate_actions_to_take()
            execute_actions("HK")

        # check on Hong Kong Stocks
        if HK_open < now and now < HK_close:
            live_data = pull_live_data()
            intraday_trader("HK", live_data)
            
            # Pull minute data to check whether to liquidate positions
            
        if HK_close - timedelta(minutes = 10) < now and now < HK_close:
            # Check pending limit orders and cancel
            live_data = pull_live_data()
            check_pending_orders("HK", live_data)
            
        # before Japan market open, pull forex to calculate Z score
        if not bef_jpn_pull and now > JPN_opening(today) - timedelta(minutes = 30):
            bef_jpn_pull = True
            pull_asian_close(2)
            pull_us_close(2)
            pull_forex_data()
            populate_diff_queue()
            populate_actions_to_take()
            execute_actions("Japan")
        
        # check on Japan stocks
        if JPN_open < now and now < JPN_close:
            # Pull minute data to check whether to liquidate positions
            live_data = pull_live_data()
            intrady_trader("Japan", live_data)
            
        if JPN_close - timedelta(minutes = 10) < now and now < JPN_close:
            # Check pending limit orders
            live_data = pull_live_data()
            check_pending_orders("Japan",live_data)
        # before Australia market open, pull forex to calculate Z score            
        if not bef_aus_pull and now > AUS_opening(today) - timedelta(minutes = 30):
            bef_aus_pull = True
            pull_asian_close(2)
            pull_us_close(2)
            pull_forex_data()
            populate_diff_queue()
            populate_actions_to_take()
            execute_actions("Australia")
        
        # check on Australian Stocks
        if AUS_open < now and now < AUS_close:
            # Pull minute data to check whether to liquidate positions
            live_data = pull_live_data()
            check_pending_orders("Australia",live_data)
            
        if AUS_open - timedelta(minutes = 10) < now and now < AUS_close:
            # Check pending limit orders   
            live_data = pull_live_data()
            check_pending_orders("Japan",live_data)

        ib.sleep(120) # Loop every two minutes
        now = datetime.now()

2021-04-27
df_forex initialized
2021-04-27 17:56:49.529902
2021-04-27 17:59:27.409866
2021-04-27 18:01:27.407657
2021-04-27 18:03:27.406211
2021-04-27 18:05:27.410619
2021-04-27 18:07:27.413614
2021-04-27 18:09:27.443163
2021-04-27 18:11:27.450238
2021-04-27 18:13:27.459030
2021-04-27 18:15:27.459157
2021-04-27 18:17:27.455930
